In [1]:
import operator
import numpy as np
from sklearn.model_selection import KFold
import gensim
from gensim.models.keyedvectors import KeyedVectors
from gensim.scripts.glove2word2vec import glove2word2vec

C:\ProgramData\Anaconda3\lib\site-packages\gensim\utils.py:862: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")


In [2]:
kf = KFold(n_splits=3, shuffle=True)

In [3]:
glove_model = KeyedVectors.load_word2vec_format("D:/wiki.de.vec", binary=False)

In [4]:
glove_model.init_sims(replace=True)

In [5]:
s = open("C:/Users/1/Desktop/alternative.txt", "r", encoding="utf-8")
lines = s.readlines()

In [6]:
#with suffixes
arr = []
for i in range(len(lines)):
    line = lines[i].strip("\n")
    items = line.split("/")
    tl = []
    for item in items[1:8]:
        tl.append(item)
    tl.append(items[-1])
    tl = tuple(tl)
    arr.append(tl)

In [7]:
dict1 = {z:arr.count(z) for z in arr}

In [8]:
dict2 = {}
for k,v in dict1.items():
    if (k[0], k[1], k[2], k[3], k[4], k[5], k[6]) not in dict2.keys():
        dict2[(k[0], k[1], k[2], k[3], k[4], k[5], k[6])] = [(k[7], v)]
    else:
        dict2[(k[0], k[1], k[2], k[3], k[4], k[5], k[6])].append((k[7], v))

In [9]:
dict3 = {}
for k,v in dict2.items():
    all_count = 0
    for x in v:
        all_count += x[1]
    dict3[k] = {'N': 0, 'NN' : 0, 'AN': 0}
    for x in v:
        dict3[k][x[0]] = x[1] / all_count

In [10]:
def decapitalize(word):
    if len(word) > 1:
        word = word[0].lower() + word[1:]
    return word

In [11]:
def all_dist(vector_1,vector_2):
    cur_dist = 0
    for i in range((len(vector_1))):
        if i == 0 or i == 3:
            try:
                ith_dist = glove_model.similarity(decapitalize(vector_1[i]), decapitalize(vector_2[i]))
                cur_dist = (1-ith_dist)*(1-ith_dist)
            except:
                ith_dist = 0
                cur_dist = (1-ith_dist)*(1-ith_dist)
        else:
            if vector_1[i] != vector_2[i]:
                cur_dist += 1
    return cur_dist

In [12]:
matrixX = []
matrixY = []
for k,v in dict3.items():
    matrixX.append(k)
    matrixY.append([v['AN'], v['N'], v['NN']])

In [13]:
matrixX[:3]

[('Engel', '<NN>', 'no', 'Bild', '<+NN>', 'no', 's'),
 ('Wolke', '<NN>', 'no', 'Schatten', '<+NN>', 'no', 'n'),
 ('Sonne', '<NN>', 'no', 'Strahl', '<+NN>', 'no', 'n')]

In [14]:
matrixY[:3]

[[0, 1.0, 0], [0, 0.5, 0.5], [0, 0, 1.0]]

In [15]:
X = np.array(matrixX)
y = np.array(matrixY)

In [16]:
kf.get_n_splits(X)

3

In [17]:
#counting accuracy
acc_fin = []
for train_index, test_index in kf.split(X):
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = y[train_index], y[test_index]
    res_all = 0
    for i in range(len(X_test)):
        distances = []
        for x in range(len(X_train)):
            distance = all_dist(X_train[x], X_test[i])
            d_t = (distance, list(y_train[x]))
            distances.append(d_t)
        distances_sort = sorted(distances)
        types_sort = np.array([0., 0., 0.])
        for item in distances_sort[:3]:
            types_sort += np.array(item[1])
        mean = types_sort / 3
        res = 0
        for k in range(len(mean)):
            res += min(mean[k], y_test[i][k])
        res_all += res
    acc = res_all/len(X_test)
    acc_fin.append(acc)
print (sum(acc_fin)/len(acc_fin))

0.444285108923
